In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

# 실제 접속은 포함하지 않음 — 샘플 형식만 유지
def get_connection():
    """
    🔒 서버 보안 보호를 위해 공개 레포에서는 실제 연결 로직을 제거했습니다.
    """
    pass

# 🔹 데이터 가져오기 예시
df_logs = pd.read_sql("SELECT * FROM reading_logs;", conn)
df_goals = pd.read_sql("SELECT * FROM reading_goals;", conn)

print(df_logs.head())
print(df_goals.head())

# 🔹 종료
conn.close()
server.stop()
print("연결 종료")


In [ ]:
# read_at에서 요일, 시간대 추출
df_logs['read_at'] = pd.to_datetime(df_logs['read_at'])
df_logs['hour'] = df_logs['read_at'].dt.hour
df_logs['weekday'] = df_logs['read_at'].dt.day_name()

# 사용자별 평균 독서 시간/패턴 (추후 한달단위, 주단위로 변경하여 사용)
user_patterns = df_logs.groupby("user_id").agg({
    "minutes_read": "mean",
    "pages_read": "mean",
    "hour": lambda x: x.mode()[0],  # 가장 많이 읽는 시간대
}).reset_index()

print(user_patterns.head())

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# --- 1️⃣ CSV 또는 DB에서 데이터 로드 (예시용: df_logs, df_goals 이미 불러온 상태라고 가정) ---

# --- 2️⃣ 목표 달성률 계산 (책, 리뷰, 시간의 평균) ---
df_goals["success_rate"] = (
    df_goals[["book_progress", "review_progress", "time_progress"]]
    .mean(axis=1)
    .fillna(0)
)

# --- 3️⃣ 사용자 단위 예측 함수 ---
def recommend_goals_for_user(user_id: int, df_goals: pd.DataFrame):
    df_user = df_goals[df_goals["user_id"] == user_id]

    recommendations = {}

    if len(df_user) < 2:
        print(f"[유저 {user_id}] 데이터가 부족합니다. (최소 2개 필요)")
        return None

    # --- (1) 시간 기반 목표 예측 ---
    X_time = df_user[["target_minutes"]]
    y_time = df_user["completed_minutes"]

    model_time = LinearRegression().fit(X_time, y_time)
    predicted_minutes = model_time.predict([[300]])[0]
    recommendations["minutes"] = int(predicted_minutes)

    # --- (2) 책 기반 목표 예측 ---
    X_books = df_user[["target_books"]]
    y_books = df_user["completed_books"]

    model_books = LinearRegression().fit(X_books, y_books)
    predicted_books = model_books.predict([[5]])[0]
    recommendations["books"] = int(predicted_books)

    # --- (3) 리뷰 기반 목표 예측 ---
    X_reviews = df_user[["target_reviews"]]
    y_reviews = df_user["completed_reviews"]

    model_reviews = LinearRegression().fit(X_reviews, y_reviews)
    predicted_reviews = model_reviews.predict([[3]])[0]
    recommendations["reviews"] = int(predicted_reviews)

    print(f"[유저 {user_id}] 다음 달 추천 목표:")
    print(f"📘 책 {recommendations['books']}권, 🕒 {recommendations['minutes']}분, ✍️ 리뷰 {recommendations['reviews']}개")
    return recommendations



In [ ]:

# --- 4️⃣ 전체 사용자 반복 ---
user_ids = df_goals["user_id"].unique()
recommendation_results = {}

for uid in user_ids:
    rec = recommend_goals_for_user(uid, df_goals)
    if rec:
        recommendation_results[uid] = rec

# --- 5️⃣ 결과 출력 ---
print("\n✅ 전체 추천 결과:")
for uid, rec in recommendation_results.items():
    print(f"User {uid} → {rec}")


In [ ]:

df_logs["hour"] = pd.to_datetime(df_logs["created_at"]).dt.hour
df_logs["weekday"] = pd.to_datetime(df_logs["created_at"]).dt.day_name()
user_pattern = df_logs.groupby(["user_id", "weekday", "hour"]).size().reset_index(name="count")
print(user_pattern)

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# -------------------------
# 전처리
# -------------------------
def preprocess_logs(df_logs):
    """ read_at 컬럼을 datetime으로 변환하고 요일/시간/주말 플래그, pages_per_min 계산 """
    df = df_logs.copy()
    # 보장: 컬럼명이 'read_at' 혹은 'created_at'인 경우 처리
    if 'read_at' in df.columns:
        df['read_at'] = pd.to_datetime(df['read_at'], errors='coerce')
    elif 'created_at' in df.columns:
        df['read_at'] = pd.to_datetime(df['created_at'], errors='coerce')
    else:
        raise ValueError("read_at 또는 created_at 컬럼 필요")

    df = df.dropna(subset=['read_at'])  # 날짜 없는 행 제거
    df['hour'] = df['read_at'].dt.hour
    df['weekday'] = df['read_at'].dt.day_name()  # Monday, Tuesday...
    df['is_weekend'] = df['read_at'].dt.weekday >= 5
    # 안전: minutes_read, pages_read 존재 시 계산
    df['minutes_read'] = pd.to_numeric(df.get('minutes_read', 0), errors='coerce').fillna(0).astype(float)
    df['pages_read'] = pd.to_numeric(df.get('pages_read', 0), errors='coerce').fillna(0).astype(float)
    # pages per minute (속도) — 0으로 나누는 것 방지
    df['ppm'] = df.apply(lambda r: (r['pages_read'] / r['minutes_read']) if r['minutes_read'] > 0 else 0.0, axis=1)
    return df

# -------------------------
# 규칙 기반 시간 추천 (개별 사용자)
# -------------------------
def rule_based_time_recommendation(df_logs_user, min_sessions=3):
    """
    입력: 한 유저의 로그 DataFrame (preprocess_logs 결과 필터된 행)
    출력: dict { 'preferred_period': 'morning/afternoon/evening/night', 'hour': int, 'session_minutes': int, 'recommended_days_per_week': int }
    """
    if df_logs_user.shape[0] < min_sessions:
        # 데이터 부족 -> 기본 권장값 반환 (예: 평일 저녁 20분, 주3회) => 콜드 스타트 문제 나중에 앱에서 회원가입시 설문으로 보완 가능
        return {'reason': 'cold_start', 'hour': 20, 'preferred_period': 'evening', 'session_minutes': 20, 'days_per_week': 3}

    # 시간대 정의 나중에 시간대 더 확장
    def period_of_hour(h):
        if 5 <= h < 11: return 'morning'
        if 11 <= h < 15: return 'afternoon'
        if 15 <= h < 19: return 'late_afternoon'
        if 19 <= h < 23: return 'evening'
        return 'night'

    df_logs_user['period'] = df_logs_user['hour'].apply(period_of_hour)

    # 가장 빈도 높은 (주중/주말 포함) 시간대, 시간
    top_period = df_logs_user['period'].mode().iloc[0]
    top_hour = int(df_logs_user['hour'].mode().iloc[0])

    # 평균 세션 길이 (분)
    avg_minutes = df_logs_user['minutes_read'].median() if df_logs_user['minutes_read'].median() > 0 else df_logs_user['minutes_read'].mean()
    if pd.isna(avg_minutes) or avg_minutes == 0:
        avg_minutes = 20  # 기본값

    # 권장 세션: 현재 평균보다 살짝 높게 (성취 가능하도록 +10%)
    session_minutes = int(max(5, round(avg_minutes * 1.1)))

    # 주당 세션 빈도: 최근 4주간 세션 카운트 기반으로 권장 (최소 1, 최대 7)
    df_logs_user['week'] = df_logs_user['read_at'].dt.isocalendar().week
    recent_weeks = df_logs_user.groupby('week').size().sort_index(ascending=False).head(4)
    if len(recent_weeks) == 0:
        days_per_week = 3
    else:
        days_per_week = int(round(recent_weeks.mean()))
        days_per_week = max(1, min(7, days_per_week))

    return {
        'reason': 'rule_based',
        'preferred_period': top_period,
        'hour': top_hour,
        'session_minutes': session_minutes,
        'days_per_week': days_per_week
    }

# -------------------------
# 사용자 군집화 (KMeans)
# -------------------------
def build_user_features_for_clustering(df_logs):
    """
    사용자별 특징행렬 생성:
    - avg_hour, median_minutes, sessions_per_week, weekend_ratio
    """
    df = df_logs.copy()
    # sessions per week
    df['week'] = df['read_at'].dt.isocalendar().week
    sessions_per_week = df.groupby(['user_id', 'week']).size().groupby('user_id').mean().rename('sessions_per_week')
    avg_hour = df.groupby('user_id')['hour'].median().rename('avg_hour')
    median_minutes = df.groupby('user_id')['minutes_read'].median().rename('median_minutes')
    weekend_ratio = df.groupby('user_id')['is_weekend'].mean().rename('weekend_ratio')  # 0~1

    features = pd.concat([avg_hour, median_minutes, sessions_per_week, weekend_ratio], axis=1).fillna(0)
    return features.reset_index()

def cluster_users(df_logs, n_clusters=3, random_state=42):
    """
    반환: (user_features_df, kmeans_labels)
    user_features_df에 'cluster' 컬럼 추가하여 반환
    """
    feats = build_user_features_for_clustering(df_logs)
    scaler = StandardScaler()
    X = scaler.fit_transform(feats[['avg_hour','median_minutes','sessions_per_week','weekend_ratio']])
    # KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
    labels = kmeans.fit_predict(X)
    feats['cluster'] = labels
    return feats, kmeans
# -------------------------
# 월간/연간 리포트 함수
# -------------------------
def monthly_report(df_goals, df_logs, year=None, month=None):
    """
    df_goals: 목표 테이블 (target_minutes, completed_minutes, target_books, completed_books 등)
    df_logs: 로그 테이블
    year, month = 특정 기간을 지정하면 해당 월 리포트(없으면 전체 집계)
    반환: user별 리포트 DataFrame
    """
    goals = df_goals.copy()
    logs = df_logs.copy()

    if year is not None:
        goals = goals[goals['year']==year]
        logs = logs[logs['read_at'].dt.year == year]
    if month is not None:
        goals = goals[goals['month']==month]
        logs = logs[logs['read_at'].dt.month == month]

    # user별 총 시간, 책수
    agg_logs = logs.groupby('user_id').agg(total_minutes=('minutes_read','sum'),
                                           sessions=('log_id','count'),
                                           avg_minutes=('minutes_read','median')).reset_index()
    agg_goals = goals.groupby('user_id').agg(target_minutes=('target_minutes','sum'),
                                             completed_minutes=('completed_minutes','sum'),
                                             target_books=('target_books','sum'),
                                             completed_books=('completed_books','sum')).reset_index()

    report = pd.merge(agg_logs, agg_goals, on='user_id', how='outer').fillna(0)
    # 성취율
    report['time_success_rate'] = report.apply(lambda r: (r['completed_minutes']/r['target_minutes']) if r['target_minutes']>0 else np.nan, axis=1)
    report['book_success_rate'] = report.apply(lambda r: (r['completed_books']/r['target_books']) if r['target_books']>0 else np.nan, axis=1)
    return report

# -------------------------
# 독서 중단 감지
# -------------------------
def detect_inactivity(df_logs, threshold_days=5, as_of=None):
    """
    사용자별 최근 read_at 확인해서 마지막 독서일로부터 threshold_days 초과하면 경고
    반환: DataFrame columns = ['user_id', 'days_since_last_read', 'inactive']
    """
    df = df_logs.copy()
    if as_of is None:
        as_of = pd.Timestamp.now()
    last = df.groupby('user_id')['read_at'].max().reset_index().rename(columns={'read_at':'last_read'})
    last['days_since_last_read'] = (as_of - last['last_read']).dt.days
    last['inactive'] = last['days_since_last_read'] >= threshold_days
    return last[['user_id','last_read','days_since_last_read','inactive']]

# -------------------------
# 개인화 미션 추천 (목표 수립)
# -------------------------
def recommend_weekly_mission(df_logs_user, df_goals_user=None):
    """
    간단한 개인화 미션:
    - 최근 4주 평균 주간 독서 분량을 계산해서 +10% 권장 (최대 상한선/하한선 적용)
    - 만약 최근 목표(월간)가 있으면, 목표 달성률 기반으로 난이도 조정
    반환: dict {'recommended_weekly_minutes': int, 'rationale': str}
    """
    df = df_logs_user.copy()
    if df.shape[0] == 0:
        return {'recommended_weekly_minutes': 3*20, 'rationale': 'cold_start_default'}  # 예: 주3회*20분

    # 주별 총합 계산 (최근 4주)
    df['week'] = df['read_at'].dt.isocalendar().week
    weekly = df.groupby('week')['minutes_read'].sum().sort_index(ascending=False).head(4)
    if weekly.empty:
        base = df['minutes_read'].median() * 3  # 임의 fallback
    else:
        base = int(round(weekly.mean()))

    # +10% 도전 제안
    recommended = int(round(base * 1.1))
    # 만약 목표(월간) 정보가 있다면 영향 반영
    if df_goals_user is not None and not df_goals_user.empty:
        # 최근 목표의 달성률 평균 사용
        df_goals_user = df_goals_user.copy()
        df_goals_user['success_rate'] = df_goals_user.apply(lambda r: (r['completed_minutes']/r['target_minutes']) if r['target_minutes']>0 else np.nan, axis=1)
        recent_success = df_goals_user['success_rate'].dropna().tail(3).mean() if not df_goals_user['success_rate'].dropna().empty else np.nan
        if not np.isnan(recent_success):
            if recent_success < 0.6:
                # 성취율 낮으면 권장량을 낮춤
                recommended = max(10, int(round(base * 0.9)))
                rationale = f'low_success_rate({recent_success:.2f})_reduce'
            elif recent_success > 0.9:
                recommended = int(round(base * 1.2))
                rationale = f'high_success_rate({recent_success:.2f})_increase'
            else:
                rationale = f'avg_success_rate({recent_success:.2f})_small_inc'
        else:
            rationale = 'no_goal_info'
    else:
        rationale = 'no_goal_info'

    # bounds
    recommended = max(10, min(2000, recommended))
    return {'recommended_weekly_minutes': recommended, 'rationale': rationale}

# -------------------------
# 유틸: 전체 사용자에게 추천/리포트 적용 예시
# -------------------------
def compute_all_recommendations(df_logs, df_goals):
    """
    입력: raw logs, raw goals (둘 다 전처리 필요)
    출력: dict with user->recommendations, clusters, reports, inactivity
    """
    logs = preprocess_logs(df_logs)
    goals = df_goals.copy()
    # ensure read_at in logs
    users = logs['user_id'].unique().tolist()

    # Cluster users
    features, kmeans = cluster_users(logs, n_clusters=3)
    cluster_map = features.set_index('user_id')['cluster'].to_dict()

    recs = {}
    reports = monthly_report(goals, logs)  # 전체 리포트 (기간 지정 가능)
    inactivity = detect_inactivity(logs)

    for uid in users:
        df_user_logs = logs[logs['user_id']==uid]
        df_user_goals = goals[goals['user_id']==uid] if 'user_id' in goals.columns else pd.DataFrame()
        rule_rec = rule_based_time_recommendation(df_user_logs)
        mission = recommend_weekly_mission(df_user_logs, df_user_goals)

        recs[uid] = {
            'rule_recommendation': rule_rec,
            'mission_recommendation': mission,
            'cluster': int(cluster_map.get(uid, -1)),
            'inactivity': inactivity[inactivity['user_id']==uid].to_dict('records')
        }

    return {'recommendations': recs, 'report_df': reports, 'cluster_features': features, 'inactivity_df': inactivity}


In [ ]:

# 예: df_logs = pd.read_sql("SELECT * FROM reading_logs", conn)

results = compute_all_recommendations(df_logs, df_goals)

# 특정 사용자 추천 확인
uid = 13
print(results['recommendations'][uid])

# 전체 리포트 보기
print(results['report_df'].head())

# 군집 결과 확인
print(results['cluster_features'].head())